In [1]:
import sys
sys.path.append('../')
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
import ipywidgets as widgets
import bqplot

from deepracer.tracks import TrackIO, Track

from deepracer.logs import \
    AnalysisUtils as au, \
    SimulationLogsIO as slio, \
    EvaluationUtils as eu, \
    PlottingUtils as pu,\
    NewRewardUtils as nr \

from reward import AWSReward, RewardFactory
import reward.awsreward as rewards
from utils.loader import load_logs, load_sample_complete_laps
from utils.transformer import df_to_params, calculate_iteration, new_names_df

tu = TrackIO(base_path='../tracks')
track = tu.load_track("reinvent_base")

def calculate_duration(df):
    df['duration'] = df[['episode','numeric_timestamp']].groupby('episode').transform(lambda x: x - x.min())

def get_df(fname):
    full_df = load_logs(fname)
    calculate_iteration(full_df)
    calculate_duration(full_df)
    return full_df

import pathlib
paths = [(x.name,x) for x in pathlib.Path('../logs').glob('*.log')]
file = widgets.Dropdown(options=paths,value=paths[0][1])

Loaded 71 waypoints


In [2]:
from traitlets import HasTraits,Instance,link
from traittypes import DataFrame

class App(HasTraits):
    df = DataFrame(default_value=None,allow_none=True).tag(sync=True)
    file = Instance(pathlib.Path,allow_none=True,default_value=None)
    
    def __init__(self):
        self.observe(self._update_file,names='file')
    
    def _update_file(self,change):
        new_file = change['new']
        self.df = get_df(new_file)
        
appModel = App() 
appModel.file = file.value
link((appModel,'file'),(file,'value'))

/home/sasha/miniconda3/envs/deepracer-viz/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: metadata {'dtype': None} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  """


In [3]:
from bqplot import Figure, LinearScale, Scatter, Lines, Axis, Tooltip,ColorScale,ColorAxis
import bqplot.pyplot as blt

In [4]:
def get_info_by_episode(df,episode):
    return df.loc[df.episode == episode, :].sort_values('step')

def calculate_instantaneous_velocity(t,x,y):
    import scipy.linalg as la
    dx, dy, dt = numpy.diff([x,y,t],prepend=0)  
    v = la.norm([dx,dy],axis=0) / dt 
    return v

def add_velocity(ep):
    x,y,t = ep[['x','y','numeric_timestamp']].values.T
    v = calculate_instantaneous_velocity(t,x,y)
    ep['velocity'] = v    
    return t,x,y,v

In [5]:
scales={'x': LinearScale(), 'y': LinearScale(),'color': ColorScale(scheme='YlGnBu')}

def create_trajectory(x,y,scales):
    tooltip = Tooltip(fields=['x', 'y', 'color' ], formats=['.2f', '.2f','s'])
    return [ Lines(x=x, y=y, scales=scales ), Scatter(x=x, y=y, scales=scales, tooltip=tooltip ) ]

def plot_track(track, scales):
    
    def plot_line(points, scales):
        x,y = points.T
        return Lines(x=x, y=y, scales=scales,colors=['gray'])

    def create_line(track,name,scales):
        points = getattr(track, name)
        return plot_line(points, scales)

    def create_track(track, scales):    
        return [ create_line(track,name,scales)
             for name in ['inner_border', 'outer_border','center_line']]

    def create_axes():
        return [ Axis(scale=scales['x'], label='x'), 
                 Axis(scale=scales['y'], orientation='vertical', label='y'),
                 ColorAxis(scale=scales['color'], label='', side='right', orientation='vertical')]

    marks = create_track(track,scales)  
    axes = create_axes()
    fig = Figure(marks=marks, axes=axes, title='L')
    fig.min_aspect_ratio = 1
    fig.max_aspect_ratio = 1
    return fig

In [6]:
ep = get_info_by_episode(appModel.df,0)
t,x,y,v = add_velocity(ep)
trajectory = create_trajectory(x,y,scales)

track_plot = plot_track(track,scales)
track_plot.marks = track_plot.marks + trajectory

In [7]:
episodes = numpy.unique(appModel.df.episode.values)
columns = ep.select_dtypes('number').columns.values
col = widgets.Dropdown(options=columns)
dd = widgets.Dropdown(options=episodes)
sl = widgets.IntSlider(min=min(episodes),max=max(episodes))
widgets.link((dd,'value'),(sl,'value'))
app = widgets.VBox(children=[file, widgets.VBox([widgets.HBox([dd,sl,col]), track_plot]) ], titles=['Choose file','App'])

In [8]:
def update_episodes(change):
    episodes = numpy.unique(appModel.df.episode.values)
    dd.options = episodes
    sl.min=min(episodes)
    sl.max=max(episodes)

def select_episode(change):
    df = appModel.df
    episode = change['new']
    ep = get_info_by_episode(df,episode)
    add_velocity(ep)
    color = ep[col.value]
    trajectory = create_trajectory(ep.x,ep.y,scales)
    trajectory[-1].color = color
    track_plot.marks = track_plot.marks[:-2] + trajectory
    
def select_column(change):
    new_column = change['new']
    df = appModel.df
    ep = get_info_by_episode(df,sl.value)
    add_velocity(ep)
    scatter = track_plot.marks[-1]
    scatter.color = ep[new_column]
    color_scale = scatter.scales['color']
    
    color_scale.min = float(scatter.color.min(initial=0.0))
    color_scale.max = float(scatter.color.max(initial=1.0)) 
    scatter.tooltip.labels = ['x','y',new_column]
    
col.observe(select_column,names='value')
sl.observe(select_episode,names='value')
appModel.observe(update_episodes,names='file')

In [9]:
app